# Integração de Gauss

Uma abordagem bem genérica para cálculo de integrais é adotar a seguinte estratégia:

$$
\int_a^b f(x)\:dx \approx \sum_{i=1}^n c_i f(x_i)
$$

Onde $x_i$ são nós e $c_i$ são os pesos correspondentes. A primeira questão é como calcular estes pesos $c_i$. 

Usando interpolação polinomial, a partir dos nós $x_i$, *sempre* se pode construir uma interpolação de Lagrange, ou seja dados um polinômio qualquer $p_{n-1}(x)$ de grau n-1, 

$$
p_n(x) = a_0 + a_1 x + a_2 x^2 + \cdots + a_{n-1} x^{n-1} = \sum_{i=1}^{n} p_{n-1}(x_i) h_i(x)
$$

Nesta equação, $h_i(x)$ é o i-ésimo interpolador de Lagrange. A partir desta aproximação polinomial, a integral pode ser calculada:

$$
\int_a^b f(x)\:dx \approx \int_a^b p_{n-1}(x)\:dx = \int_a^b \left[\sum_{i=1}^{n} p_{n-1}(x_i) h_i(x)\right] \: dx = \sum_{i=1}^{n} \int_a^b p_{n-1}(x_i) h_i(x)\:dx
$$

Se o polinômio $p_{x-1}(x)$ interpola a função $f(x)$ nos n nós $x_i$, 

$$
\sum_{i=1}^{n} \int_a^b p_{n-1}(x_i) h_i(x)\:dx = \sum_{i=1}^{n} \int_a^b f(x_i) h_i(x)\:dx = \sum_{i=1}^n c_i f(x_i)
$$

com

$$
c_i = \int_a^b h_i(x)\:dx \qquad 1 \le i \le n
$$


**Os pontos $x_i$ são fixados por quem quer calcular a integral

<!-- TEASER_END -->

E se escolhermos pontos óptimos de integração, será que conseguimos algo melhor???

Para facilitar as coisas, as integrais serão sempre calculadas em um domínio $-1\le x \le 1$. Para um domínio qualquer, 

$$
\int_a^b f(x)\:dx = \int_{-1}^1 f\left[x(\xi)\right] J(\xi) \:d\xi
$$

onde $J(\xi)$ é o Jacobiano da transformação e uma transformação do tipo 

$$
x(\xi) = \frac{1-\xi}{2} a + \frac{1+\xi}{2} b
$$

de modo que

$$
J(\xi) = \frac{dx}{d\xi} = \frac{b-a}{2}
$$

A idéia dop método de Gauss, é encontrar os coeficientes $c_i$ e $x_i$ que maximizam o grau do polinomio interpolador que pode ser integrado *exatamente*:

$$
\int_{-1}^1 f(x) \:dx = c_1 f(x_1) + c_2 f(x_2)
$$

Neste caso, existem 4 graus de liberdade: $c_1$, $c_2$, $x_1$ e $x_2$. Um polinômio de grau 3 têm 4 coeficientes. Assim, 

$$
\int_{-1}^1 \left(a_0 + a_1x + a_2x^2 + a_3x^3\right)\:dx = a_0\int_{-1}^1 1\:dx + 
a_1 \int_{-1}^1x \:dx + a_2\int_{-1}^1 x^2\:dx + a_3 \int_{-1}^1 x^3 \:dx 
$$

Integrando exatamente os polinômios 1, $x$, $x^2$ e $x^3$, temos:

$$
c_1\cdot 1 + c_2\cdot 2 = \int_{-1}^1 1\:dx  = 2\\
c_1\cdot x_1 + c_2\cdot x_2 = \int_{-1}^1 x\:dx  = 0\\
c_1\cdot x_1^2 + c_2\cdot x_2^2 = \int_{-1}^1 x^2\:dx  = \frac{2}{3}\\
c_1\cdot x_1^3 + c_2\cdot x_2^3 = \int_{-1}^1 x^3\:dx  = 0\\
$$

Isto é um sistema de equações algébricas de 4 equações e 4 incógnitas cuja solução é:

$$
c_1 = 1, \quad c_2=1, \quad x_1 = -\frac{\sqrt{3}}{3}, \quad x_1 = \frac{\sqrt{3}}{3}
$$

Com estes pesos e nós, integrais de grau 3 ou inferior são calculados exatamente!



# Generalização do processo anterior

Este processo poderia ser repetido para polinômios de grau mais alto. Ou eventualmente usando outros tipos de função, fixando alguns nós ou até mesmo fixando alguns pesos. Mas existem formas alternativas de obter os nós e pesos.

A biblioteca `Jacobi` <https://github.com/pjabardo/Jacobi.jl> calcula os pesos e nós para as seguintes integrais:

$$
\int_{-1}^1 (1-x)^\alpha (1+x)^\beta f(x)\:dx = \sum_{i=1}^n w_i^{\alpha,\beta} f(x_i)
$$

In [ ]:
using Jacobi
using PyPlot
using Polynomials

# Exemplos



In [ ]:
function gaussxw(n, α, β, ::Type{T}=Float64) where {T<:Number}
    z = zgj(n, α, β, T)
    w = wgj(z, α, β)
    
    return z, w
end
applyint(g, f) = sum(f.(g[1]) .* g[2])

In [ ]:
function calcerr(g, f, Ie)
    
    I = applyint(g, f)
    return abs(I-Ie)
end

In [ ]:
setprecision(BigFloat, 2048)

In [ ]:
nn1 = 2:25
gc1 = gaussxw.(nn1, 0, 0);

In [ ]:
nn2 = 2:40
gc2 = gaussxw.(nn2, 0, 0, BigFloat);

## $1 + x + x^2 + x^3 + x^4 + x^5$

In [ ]:
coefs = [1,1,1,1,1,1]
P1 = Poly(Float64.(coefs))
IP1 = polyint(P1)
Ie1 = IP1(1) - IP1(-1)

In [ ]:
err1 = calcerr.(gc1, x->P1(x), Ie1);

In [ ]:
loglog(nn1, err1, "bo")

# Polynomio de grau 19 com coeficientes aleatórios

In [ ]:
coefs = randn(20)
P2 = Poly(Float64.(coefs))
IP2 = polyint(P2)
Ie2 = IP2(1) - IP2(-1)

In [ ]:
err2 = calcerr.(gc1, x->P2(x), Ie2);

In [ ]:
loglog(nn1, err2, "bo")


## $\sin nx$

$$
\int_{-1}^1 \sin n x \:dx = \frac{2\sin n}{n}
$$

In [ ]:
n=4
f(x,n, ::Type{T}=Float64) where T = cos(n*x)
ife(n, ::Type{T}=Float64) where T = 2*sin(n*one(T))/n

xx = -1:0.01:1
plot(xx, f.(xx,n), "k-")


In [ ]:
n = 3
Ie3 = ife(n)
err3 = calcerr.(gc1, x->f(x,n), Ie3) 
Ie3b = ife(n, BigFloat)
err3b = calcerr.(gc2, x->f(x,n,BigFloat), Ie3b) 


loglog(nn2, err3b, "rs")
loglog(nn1, err3, "bo")


## $\int_{-1}^1 (1 + x + x^2 + x^3 + x^4 + x^5)(1-x)^2(1+x)^3 \:dx$

### Gauss normal

In [ ]:
P4 = Poly([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])
f4 = P4 * Poly([1.0, -1])^2 * Poly([1.0, 1.0])^3

IP4 = polyint(f4)
Ie4 = IP4(1.0) - IP4(-1.0)


In [ ]:
err4 = calcerr.(gc1, x->f4(x), Ie4);

In [ ]:
loglog(nn1, err4, "bo")

### Usando os pesos - Gauss-Jacobi



In [ ]:
gc3 = gaussxw.(nn1, 2, 3);

In [ ]:
err4b = calcerr.(gc3, x->P4(x), Ie4);

In [ ]:
lgj = loglog(nn1, err4, "bo", label="Gauss")
lgjab = loglog(nn1, err4b, "rs", label="Inclui peso")
legend()

## Mesmo polinômio mas inclui as extremidades

In [ ]:
function gaussxwL(n, α, β, ::Type{T}=Float64) where {T<:Number}
    z = zglj(n, α, β, T)
    w = wglj(z, α, β)
    
    return z, w
end

In [ ]:
gc4 = gaussxwL.(nn1, 0, 0);
gc5 = gaussxwL.(nn1, 2, 3);

In [ ]:
err5 = calcerr.(gc4, x->f4(x), Ie4);
err5b = calcerr.(gc5, x->P4(x), Ie4);

In [ ]:
lgj = loglog(nn1, err4, "bo", label="Gauss")
lgjab = loglog(nn1, err4b, "rs", label="Inclui peso")
lglj = loglog(nn1, err5, "gx", label="Gauss-Lobatto")
lgjlab = loglog(nn1, err5b, "k.", label="GL Inclui peso")
legend()

# Podemos incluir apenas uma extremidade

 * Não inclui qualquer extremidade: `zgj`, `wgj`
 * Inclui ambas as extremidades: `zglj`, `wglj`
 * Inclui apenas -1: `zgrjm`, `wgrjm`
 * Inclui apenas +1: `zgrjp`, `wgrjp`
 

## Problema 1

Calcule usando os diversos métodos apresentados a seguinte integral:

$$
\int_{-1}^1 \left(1- | x |\right) \: dx
$$



## Problema 2

Calcule utilizando os diversos métodos apresentados a seguinte integral:

$$
\int_{-1}^1 \exp\left(-x^2\right)\:dx
$$

## Problema 3


Calcule utilizando os diversos métodos apresentados a seguinte integral:

$$
\int_{-\infty}^\infty \exp\left(-x^2\right)\:dx
$$

Dicas: O jeito mais fácil é calcular esta integral de $-L$ a $L$ e ver como se comporta quando $L\longrightarrow\infty$. Mas o melhor jeito é mapear $x$ para outra variável $\xi$ de modo que quando $x\longrightarrow\infty$, $\xi\longrightarrow 1$ (ou qualquer outra constante).

## Problema 4

Em Pitometria, calcular a vazão correponde a integrar o perfil de velocidades. Deste modo, 

$$
Q = \int_0^R 2\pi r\cdot u(r) \:dr
$$

Encontre os pesos e os nós de integração usando o método de Gauss (com peso). Dica: mapeie $0\le r\le R$ para uma variável $\xi$ de modo que os limites de integração sejam $-1 \le \xi \le 1$.



## Problema 5: desafio

No problema anterior, foram utilzados polinômios. Tente encontrar os pesos e nós para sequências de funções do tipo

$$
1, \:\ln x, \:x, \:x^2, \:x^3, \:x^4, \:\text{etc}
$$